In [1]:
!pip install flask flask-cors

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import json
from query_engine_interface import (
    generate_json_with_openai_few_shot,
    retrieve_context_from_vector_db,
    setup_vector_db,
    load_csv,
    construct_graph,
)

app = Flask(__name__)
CORS(app)

# Load CSV and construct the graph
file_path = "sample_integration_data.csv"
integration_data = load_csv(file_path)
if integration_data is not None:
    graph = construct_graph(integration_data)
    print("Graph successfully constructed.")
else:
    print("Failed to load CSV or construct graph.")
    graph = None

# Ensure the vector database is set up
vector_db_client = None
if graph:
    vector_db_client = setup_vector_db(graph)
    if vector_db_client:
        print("Vector database setup completed.")
    else:
        print("Vector database setup failed.")

@app.route("/", methods=["GET"])
def home():
    """
    Root endpoint for debugging or displaying server status.
    """
    return jsonify({"message": "Server is running. Use POST /query to interact."}), 200

@app.route("/query", methods=["POST"])
def query_interactions():
    """
    Endpoint to retrieve system interactions for a given query.
    """
    if vector_db_client is None:
        return jsonify({"error": "Vector database not available"}), 503

    data = request.get_json()
    query = data.get("query", "")

    if not query:
        return jsonify({"error": "Query parameter is missing"}), 400

    # Retrieve context
    retrieved_context = retrieve_context_from_vector_db(vector_db_client, query)
    if not retrieved_context:
        return jsonify({"error": "No context found"}), 404

    # Generate JSON response using OpenAI
    json_response = generate_json_with_openai_few_shot(retrieved_context, query)
    if json_response:
        try:
            # Parse the JSON string into a Python object
            parsed_response = json.loads(json_response)
            return jsonify({"query": query, "response": parsed_response}), 200
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            return jsonify({"error": "Failed to parse JSON response"}), 500
    else:
        return jsonify({"error": "Failed to generate JSON response"}), 500

if __name__ == "__main__":
    print("Starting Flask server on http://127.0.0.1:5000. Use Ctrl+C to stop the server.")
    app.run(port=5000, debug=True, use_reloader=False)

CSV loaded successfully!
Graph successfully constructed.
Vector database populated with 100 points!
Vector database setup completed.
Starting Flask server on http://127.0.0.1:5000. Use Ctrl+C to stop the server.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Nov/2024 13:51:12] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 13:53:55] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 13:53:58] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 13:54:38] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 13:54:42] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 13:56:34] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 13:56:38] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 13:56:57] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 13:57:00] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 14:01:33] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 14:01:37] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 14:03:11] "OPTIONS /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 14:03:14] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 14:06:23] "OPTIONS /query HTTP/1.1" 200 